In [1]:
import pandas as pd
import re
import string
import jieba
import collections

In [49]:
all_data = pd.read_excel("./data/Herceptin_filtered.xlsx")

In [55]:
all_data.head(10)

,Case Number,Contact Type,Contact Channel,Response Channel,Enquiry Classification,Subject,Description,Product,Response,Unnamed: 9
0,1420656,Nurse,Affiliate MI mailbox,Phone,Stability/Storage/Reconstitution,beyond temperature,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配...",Herceptin,根据说明书信息，赫赛汀需要存放于2-8度的温度中，超温保存对于药品活性会产生影响，不建议使用。,NaN
1,1423956,Internal,Phone,Email,Stability/Storage/Reconstitution,beyond temperature-herceptin,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）,Herceptin,[#Responded via email: 罗氏医学信息: [01423956]#],NaN
3,1434184,Physician,Internal Webform,Email,Safety,AE-Herceptin&Thromboembolism,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）,Herceptin,[#Responded via email: 罗氏医学信息: [01434184]#],NaN
4,1436011,Patient,Phone,Phone,Product Quality,Quality-herceptin,拿到的赫赛汀呈现块状是否正常,Herceptin,由于赫赛汀是抽真空的工艺，抽掉空气后就是块状的，但由于运输过程中撞击、摇晃等因素可能会把块状...,NaN
5,1438071,Internal,Phone,Email;Phone,Product Quality,quality-herceptin,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,Herceptin,根据Herceptin说明书信息，溶解后为无色至淡黄色，澄清至微乳光溶液。该case记录PC上报,NaN
6,1439604,Internal,In person,Email,Publication/Educational Material Request,PI-herceptin,赫赛汀最新说明书,Herceptin,[#Responded via email: 罗氏医学信息: [01439604]#],NaN
7,1442983,Patient,Phone,Phone,Logistics/Funding/Regulatory,supply-herceptin,赫赛汀哪里可以购买,Herceptin,提供无锡及常州药房信息，建议事先电话联系已确认库存情况,NaN
8,1443005,Patient,Phone,Phone,Safety,AE-overdue&herceptin,有无过期赫赛汀输注到患者体内的疗效或安全性数据（患者怀疑医院将过期药品输注到体内，无产品批号信息）,Herceptin,赫赛汀需在有效期内使用，热线无过期产品用于人体的临床试验数据，建议确认是否输注过期产品，记录AE,NaN
9,1444557,Nurse,Phone,Phone,Stability/Storage/Reconstitution,beyond temperature-Herceptin,赫赛汀注射前发现稀释液有结冰情况，能否使用，产品批号（SH0128，N3834/B3051）,Herceptin,根据说明书信息，赫赛汀需要在2-8度保存和运输，超温保存会影响药品活性，从而影响疗效和安全性...,NaN
10,1444573,Patient,Phone,Phone,Stability/Storage/Reconstitution,beyond temperature-herceptin,赫赛汀用前发现稀释液有冰晶，能否使用,Herceptin,根据说明书信息，赫赛汀需在2-8度保存与运输，超温保存将影响产品活性，热线不建议使用。记录AE,NaN


In [51]:
all_data.isna().sum()

Case Number                400
Contact Type               402
Contact Channel            402
Response Channel           402
Enquiry Classification     402
Subject                    402
Description                332
Product                    402
Response                    95
Unnamed: 9                1157
dtype: int64

In [52]:
all_data.dropna(axis=0, inplace=True, subset=['Case Number', 'Contact Type'])

In [53]:
all_data.isna().sum()

Case Number                 0
Contact Type                0
Contact Channel             0
Response Channel            0
Enquiry Classification      0
Subject                     0
Description                 0
Product                     0
Response                    0
Unnamed: 9                756
dtype: int64

In [54]:
len(all_data)

756

In [56]:
all_data.drop(all_data[all_data['Contact Type'] == 'Patient'].index, inplace=True)

In [57]:
len(all_data)

201

In [58]:
# all_data.to_excel('./data/Herceptin_patient_filtered.xlsx')

In [2]:
mesh_data = pd.read_csv('./mesh_ref_copy1.csv')

In [7]:
print(len(mesh_data))
mesh_data.head()

911261


,1000065,anatomy category,A
0,1000066,organisms category,B
1,1000067,diseases category,C
2,1000068,chemicals and drugs category,D
3,1000069,"analytical, diagnostic and therapeutic techniq...",E
4,1000070,psychiatry and psychology category,F


In [10]:
def is_zh(string):
    return all('\u4e00' <= char <= '\u9fff' for char in string)

In [16]:
chinese_word_dict = []
for idx,row in mesh_data.iterrows():
    if is_zh(row['anatomy category']):
        chinese_word_dict.append(row['anatomy category'])

In [22]:
print(len(chinese_word_dict))
# chinese_word_dict

44223


In [20]:
mimc_data = pd.read_csv('./mimc_not_matched.csv')

In [25]:
for idx, row in mimc_data.iterrows():
    chinese_word_dict.append(row['zh'])
print(len(chinese_word_dict))

59002


In [29]:
cnkd_data = pd.read_csv('./cnkd_not_matched.csv')
print(len(cnkd_data))

6555


In [31]:
for idx, row in cnkd_data.iterrows():
    chinese_word_dict.append(row['zh'])
print(len(chinese_word_dict))

65557


In [32]:
chinese_word_dict = list(set(chinese_word_dict))

In [33]:
len(chinese_word_dict)

54609

In [38]:
'赫赛汀' in chinese_word_dict

True

In [39]:
import pickle
with open('./chinese_word_dict.pickle', 'wb') as f:
    pickle.dump(chinese_word_dict, f)

In [40]:
f_chinese_word_dict = open('chinese_word_dict.txt', 'w')
write_list = [word+"\n" for word in chinese_word_dict]
f_chinese_word_dict.writelines(write_list)
f_chinese_word_dict.close()